# Notebook: Filter Sentences from Reviews to create Dataset for Annotation

## Packages

In [1]:
from bs4 import BeautifulSoup
from langdetect import detect
import pandas as pd
import json
import nltk
from nltk.tokenize import sent_tokenize
import re

## Settings

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Constants

In [3]:
REVIEWS_PATH = "reviews_dataset/reviews.csv"
RANDOM_STATE = 43

## Code

### Load Dataset

In [4]:
df_reviews = pd.read_csv(REVIEWS_PATH)

In [5]:
df_reviews

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die <LOC> schmeckte leicht säuerlich, der klei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...
1,1,907168709,2005104,0,Wir sind hier nicht in Bayern!,2023-08-01,173stefanh,"München, Deutschland","Wir sind hier nicht in <LOC>, sind die freundl...",1.0,berlin,Augustiner am Gendarmenmarkt,de,"Wir sind hier nicht in Bayern, sind die freund..."
2,2,902540630,2005104,0,Heute leider zum abgewöhnen,2023-07-12,Patrick L,"Rüdersdorf, Deutschland",Ich freute mich sehr auf das Wienerschnitzel u...,3.0,berlin,Augustiner am Gendarmenmarkt,de,Ich freute mich sehr auf das Wienerschnitzel u...
3,3,895783388,2005104,0,Teils-Teils,2023-06-18,J9324NNpeterr,"Stellenbosch, Südafrika",Meine Erfahrungen in diesem Jahr waren gemisch...,4.0,berlin,Augustiner am Gendarmenmarkt,de,Meine Erfahrungen in diesem Jahr waren gemisch...
4,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",ich hätte gerne etwas zum Essen gesagt. Aber e...,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,3596,889964622,5207419,0,Nicht besonderes,2023-05-11,Mobile28959210843,"Mönchengladbach, Deutschland",Das Essen ist okay mehr aber auch nicht. Ambie...,3.0,mönchengladbach,Brauerei zum Stefanus,de,Das Essen ist okay mehr aber auch nicht. Ambie...
3597,3597,870345760,5207419,0,"Ungebührender Empfang, sehr gutes Essen",2022-12-01,Korich,"Görlitz, Deutschland",Zunächst einmal wurden wir von einem Mann mit ...,4.0,mönchengladbach,Brauerei zum Stefanus,de,Zunächst einmal wurden wir von einem Mann mit ...
3598,3598,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Wir sind nicht zum ersten Mal aber auch defini...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...
3599,3599,863841135,5207419,0,Gutes Essen mit Biergartenambiente,2022-10-09,788heinzn,"Köln, Deutschland","Gutes Essen, Portionen sehr groß (für uns zu g...",4.0,mönchengladbach,Brauerei zum Stefanus,de,"Gutes Essen, Portionen sehr groß (für uns zu g..."


### Filter Sentences

In [6]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx'])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'], language="german")
    sentence_index = 0
    for sentence in sentences:
        if len(sentence) > 1:
            new_row = row.copy() 
            new_row['text'] = sentence 
            new_row['sentence_idx'] = sentence_index
            df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
            sentence_index += 1

In [7]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die <LOC> schmeckte leicht säuerlich, der klei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,0
1,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Der Schweinebraten war lauwarm und der Kaisers...,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,1
2,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Ich bin so enttäuschst.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,2
3,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Gehe ich nicht mehr hin.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,3
4,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Schlecht kann ich auch woanders, besonders bei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15344,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Das Essen schmeckt aber generell grandios.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,1
15345,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Der Beilagensalat ist auch der Kracher: frisch...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,2
15346,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Im Stefanus gibt es auch selbstgebrautes, sehr...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,3
15347,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Der Service ist sehr nett, zuvorkommend und he...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,4


### Filter Non-German Sentences

Even if the language code = "de", we have observed that some sentences are in other languagesthan german. These are excluded as well with the help of Google's language-detection.

In [8]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df_reviews_sentences['detected_language'] = df_reviews_sentences['text'].apply(detect_language)
df_reviews_sentences[df_reviews_sentences['detected_language'] != 'de']

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,detected_language
17,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",Es war viel los.,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...,3,af
66,13,866452621,2005104,1,Mind this touri-gap,2022-10-29,452stephand,"Nürnberg, Deutschland",Fazit: mind this touri-gap,1.0,berlin,Augustiner am Gendarmenmarkt,de,Wunderschöne Location mit mangelhafter Leitung...,1,en
93,17,856900319,2005104,1,Deftig bayrisch,2022-08-28,tatrab,"Kaiserslautern, Deutschland",Kann man mal hin,3.0,berlin,Augustiner am Gendarmenmarkt,de,Vorzubereitete Speisen in bayrischer Manier zu...,2,tl
113,22,847064431,2005104,2,Enttäuschend,2022-07-09,HerSchnu,"Erfurt, Deutschland",Fing schon gut an….,2.0,berlin,Augustiner am Gendarmenmarkt,de,In jedem bayerischem Gasthof isst man besser… ...,2,en
131,25,883653345,947082,0,Tolle Burger!,2023-03-26,dittelbrunn,"Lübeck, Deutschland",Ketchup.,5.0,berlin,Burgermeister,de,Wir waren in der Filiale am S-Bahnhof Eberswal...,2,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15202,3564,866578862,13144674,6,Leckeres Essen in Mönchengladbach,2022-10-30,marcelbF2818DO,NaN,BRAVO!,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Ein tolles Restaurant! Ambiente und Empfang su...,3,pt
15231,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Klasse.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",4,et
15241,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Place to be.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",14,en
15249,3572,861638375,13144674,6,"Handwerklich einwandfrei, Hohe Qualität und ei...",2022-09-24,Tilmann S,NaN,Top.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Eher zufällig kehrten wir als Ortsunkundige im...,4,en


In [9]:
df_reviews_sentences = df_reviews_sentences[df_reviews_sentences['detected_language'] == 'de']

### Balancing

In [10]:
df_balanced_reviews_sentences = df_reviews_sentences.groupby(['rating'], group_keys=False).apply(lambda x: x.sample(700, replace=False, random_state=RANDOM_STATE))
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Randomisierung

In [11]:
df_balanced_reviews_sentences = df_balanced_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Check If Sentences are Unique

In [12]:
df_balanced_reviews_sentences.groupby(["review_id", "sentence_idx"]).size().reset_index(name="count")

,review_id,sentence_idx,count
0,845575616,0,1
1,845636774,0,1
2,845636774,1,1
3,845636774,2,1
4,845658907,3,1
...,...,...,...
3495,916485162,0,1
3496,916494891,0,1
3497,916494891,3,1
3498,916494891,4,1


### Store as .csv 

In [13]:
df_reviews_sentences.to_csv("reviews_dataset/reviews_sentences.csv")

In [14]:
df_balanced_reviews_sentences.to_csv("reviews_dataset/balanced_reviews_sentences.csv")

In [15]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,detected_language
0,1537,867609212,743219,1,Schönes Ambiente,2022-11-07,schneiderrebecca,"Würzburg, Deutschland",Das Personal freundlich.,4.0,leipzig,Cafe Riquet,de,Ich war gegen 17:30 Uhr in dem Kaffeehaus. Die...,4,de
1,2443,911273338,1342313,1,Nicht nur überaus köstlich in romantischer Atm...,2023-08-18,Michael E,NaN,"Überragender Service, köstliche Speisen!Wir ko...",5.0,bielefeld,Jivino Enoteca,de,"Ungefragt wurde uns vorab serviert, was wir mö...",1,de
2,2150,910109779,1118853,0,Einfache Biergartenküche zu gehobenen Preisen,2023-08-13,ingrid240,"Frankfurt am Main, Deutschland","Freundlicher Service, sehr gutes Bier.Das <LOC...",2.0,hannover,Bavarium,de,"Freundlicher Service, sehr gutes Bier.Das Tiro...",0,de
3,724,857521547,1190751,12,Schlemmermenü super köstlich.,2022-08-31,409gisell,NaN,Jeder Gang ein Genuss.,5.0,frankfurt,Trattoria i Siciliani,de,Jeder Gang ein Genuss. Wir haben Fisch und Fle...,0,de
4,2984,892846168,2481134,1,"Essen, Wein und Service sehr zu empfehlen",2023-05-31,Explore04799982138,"Gaiole in Chianti, Italien",Ein tolles italienisches Restaurant.,4.0,mannheim,Da Vino,de,Ein tolles italienisches Restaurant. Essen tol...,0,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,504,868718065,7235024,2,Mein erster originaler rheinischer Sauerbraten,2022-11-16,ALMSUPA,"Wien, Österreich",Danke für die tolle Zeit bei Euch!,4.0,köln,Früh am Dom,de,"Waren in der City mit dem ""einheimischen"" Cous...",5,de
3496,218,883034414,954454,2,War mit 4 Freunden hier essen. Freundliches Pe...,2023-03-21,Joker-BN,"Bonn, Deutschland",Nach Rücksprache mit dem Chef wurde eine Ausna...,4.0,hamburg,ALEX Hamburg,de,Sehr schöne Lage mit Blick auf die Alster. Gut...,14,de
3497,2205,853942330,1390048,2,"Gutes Essen, meist mit Wartezeit",2022-08-13,TobiMTravel,"München, Deutschland",Die <LOC> waren sehr gut und das Sauerkraut eb...,4.0,nürnberg,Bratwursthäusle,de,Wir waren als paar an einem Samstag Mittag dor...,1,de
3498,3175,889498235,7322855,4,Ein Reines Gedicht,2023-05-08,Tim R,NaN,Es schmeckt als ob die Kuh jeden Tag vom Bauer...,5.0,augsburg,SteakManufaktur,de,Zur Ambiente:Wir würden herzlich empfangen vom...,11,de
